In [1]:
%AddJar file:/home/jovyan/aas/ch06-lsa/target/ch06-lsa-2.0.0-jar-with-dependencies.jar

Starting download from file:/home/jovyan/aas/ch06-lsa/target/ch06-lsa-2.0.0-jar-with-dependencies.jar
Finished download of ch06-lsa-2.0.0-jar-with-dependencies.jar


In [2]:
import edu.umd.cloud9.collection.XMLInputFormat
import edu.stanford.nlp.ling.CoreAnnotations.{LemmaAnnotation, SentencesAnnotation, TokensAnnotation}
import edu.stanford.nlp.pipeline.{Annotation, StanfordCoreNLP}
import edu.umd.cloud9.collection.wikipedia.WikipediaPage
import edu.umd.cloud9.collection.wikipedia.language.EnglishWikipediaPage
import java.util.Properties

import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.io.{LongWritable, Text}
import org.apache.spark.ml.feature.{CountVectorizer, IDF}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{DataFrame, Dataset, SparkSession}

import scala.collection.JavaConverters._
import scala.collection.mutable.ArrayBuffer

val spark = SparkSession.builder().getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@32ae1ce3


i...


Spark local-1539613677795: Some(http://74107bd78b99:4040)

In [5]:
val path = "Data/Wikipedia-Geometry.xml"
@transient val conf = new Configuration()
conf.set(XMLInputFormat.START_TAG_KEY, "<page>")
conf.set(XMLInputFormat.END_TAG_KEY, "</page>")
val kvs = spark.sparkContext.newAPIHadoopFile(path, classOf[XMLInputFormat],
                                              classOf[LongWritable], classOf[Text], conf)
val rawXmls = kvs.map(_._2.toString).toDS()

path = Data/Wikipedia-Geometry.xml
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml
kvs = Data/Wikipedia-Geometry.xml NewHadoopRDD[4] at newAPIHadoopFile at <console>:66
rawXmls = [value: string]


[value: string]

In [6]:
rawXmls.show

+--------------------+
|               value|
+--------------------+
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
|<page>
    <title...|
+--------------------+
only showing top 20 rows



In [7]:
def wikiXmlToPlainText(pageXml: String): Option[(String, String)] = {
// Wikipedia has updated their dumps slightly since Cloud9 was written,
// so this hacky replacement is sometimes required to get parsing to work.
val hackedPageXml = pageXml.replaceFirst(
"<text xml:space=\"preserve\" bytes=\"\\d+\">",
"<text xml:space=\"preserve\">")
val page = new EnglishWikipediaPage()
WikipediaPage.readPage(page, hackedPageXml)
if (page.isEmpty) None
else Some((page.getTitle, page.getContent))
}
val docTexts = rawXmls.filter(_ != null).flatMap(wikiXmlToPlainText)

docTexts = [_1: string, _2: string]


wikiXmlToPlainText: (pageXml: String)Option[(String, String)]


[_1: string, _2: string]

In [8]:
docTexts.show

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|Category:Discrete...|Category:Discrete...|
|Category:Incidenc...|Category:Incidenc...|
|Category:Metric g...|Category:Metric g...|
|Category:Integral...|Category:Integral...|
|Category:Conforma...|Category:Conforma...|
|Category:Trigonom...|Category:Trigonom...|
|Category:Convex g...|Category:Convex g...|
|             Coaxial|Coaxial
 
 

In g...|
|Category:Technica...|Category:Technica...|
|   Category:Symmetry|Category:Symmetry...|
|Category:Homogene...|Category:Homogene...|
|       Ambient space|Ambient space
An ...|
|Category:Duality ...|Category:Duality ...|
|          Superspace|Superspace
Supers...|
|     Geometry Center|Geometry Center
T...|
|          Dehn plane|Dehn plane
In geo...|
|Complex reflectio...|Complex reflectio...|
|Category:Geometri...|Category:Geometri...|
|    Lipschitz domain|Lipschitz domain
...|
|         Visual hull|Visual hul

In [9]:
def createNLPPipeline(): StanfordCoreNLP = {
    val props = new Properties()
    props.put("annotators","tokenize, ssplit, pos, lemma")
    new StanfordCoreNLP(props)
}

def isOnlyLetters(str: String): Boolean = {
    str.forall(c => Character.isLetter(c))
}

def plainTextToLemmas(text: String, stopWords: Set[String],
    pipeline: StanfordCoreNLP): Seq[String] = {
    val doc = new Annotation(text)
    pipeline.annotate(doc)
    
    val lemmas = new ArrayBuffer[String]()
    val sentences = doc.get(classOf[SentencesAnnotation])
    for (sentence <- sentences.asScala;
        token <- sentence.get(classOf[TokensAnnotation]).asScala){
        val lemma = token.get(classOf[LemmaAnnotation])
        if(lemma.length > 2 && !stopWords.contains(lemma)
          && isOnlyLetters(lemma)){
          lemmas += lemma.toLowerCase
        }
    }
    lemmas
}

createNLPPipeline: ()edu.stanford.nlp.pipeline.StanfordCoreNLP
isOnlyLetters: (str: String)Boolean
plainTextToLemmas: (text: String, stopWords: Set[String], pipeline: edu.stanford.nlp.pipeline.StanfordCoreNLP)Seq[String]


In [10]:
val stopWords = scala.io.Source.fromFile("stopwords.txt").getLines().toSet
val bStopWords = spark.sparkContext.broadcast(stopWords)

val terms: Dataset[(String, Seq[String])] = 
 docTexts.mapPartitions { iter =>
     val pipeline = createNLPPipeline()
     iter.map { case(title, contents) =>
        (title, plainTextToLemmas(contents, bStopWords.value, pipeline))
     }
 }

stopWords = Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, ...


Set(down, it's, ourselves, that's, for, further, she'll, any, there's, this, haven't, in, ought, myself, have, your, off, once, i'll, are, is, his, why, too, why's, am, than, isn't, didn't, himself, but, you're, below, what, would, i'd, if, you'll, own, they'll, up, we're, they'd, so, our, do, all, him, had, nor, before, it, a, she's, as, hadn't, because, has, she, yours, or, above, yourself, herself, she'd, such, they, each, can't, don't, i, until, that, out, he's, cannot, to, we've, hers, you, did, let's, most, here, these, hasn't, was, there, when's, shan't, doing, at, through, been, over, i've, on, being, same, how, whom, my, after, who, itself, me, them, by, then, couldn't, he, should, few, wasn't, again, while, their, not, with, from, you've, they've, what's, wouldn't, both, could, its, under, which, you'd, an, be, here's, into, where, he'll, her, themselves, were, more, we'd, where's, they're, who's, between, aren't, ours, about, doesn't, how's, against, during, no, very, we, ha

In [11]:
val termsDF = terms.toDF("title","terms")

termsDF = [title: string, terms: array<string>]


[title: string, terms: array<string>]

In [12]:
val filtered = termsDF.where(size($"terms")>1)

filtered = [title: string, terms: array<string>]


[title: string, terms: array<string>]

In [13]:
filtered.show

Adding annotator tokenize
TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
Adding annotator ssplit
edu.stanford.nlp.pipeline.AnnotatorImplementations:
Adding annotator pos
Reading POS tagger model from edu/stanford/nlp/models/pos-tagger/english-left3words/english-left3words-distsim.tagger ... done [3.4 sec].
Adding annotator lemma
+--------------------+--------------------+
|               title|               terms|
+--------------------+--------------------+
|Category:Discrete...|[category, discre...|
|Category:Incidenc...|[category, incide...|
|Category:Metric g...|[category, metric...|
|Category:Integral...|[category, integr...|
|Category:Conforma...|[category, confor...|
|Category:Trigonom...|[category, trigon...|
|Category:Convex g...|[category, convex...|
|             Coaxial|[coaxial, geometr...|
|Category:Technica...|[category, techni...|
|   Category:Symmetry|[category, symmetry]|
|Category:Homogene...|[category, homoge...|
|       Ambient space|[a

In [15]:
val numTerms = 20000
val countVectorizer = new CountVectorizer().
    setInputCol("terms").
    setOutputCol("termFreqs").
    setVocabSize(numTerms)
val vocabModel = countVectorizer.fit(filtered)
val docTermFreqs = vocabModel.transform(filtered)

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma


numTerms = 20000
countVectorizer = cntVec_024dfea34f89
vocabModel = cntVec_024dfea34f89
docTermFreqs = [title: string, terms: array<string> ... 1 more field]


[title: string, terms: array<string> ... 1 more field]

In [16]:
docTermFreqs.cache()

[title: string, terms: array<string> ... 1 more field]

In [17]:
val idf = new IDF().
    setInputCol("termFreqs").
    setOutputCol("tfidfVec")
val idfModel = idf.fit(docTermFreqs)
val docTermMatrix = idfModel.transform(docTermFreqs).select("title","tfidVec")

Adding annotator tokenize
Adding annotator ssplit
Adding annotator pos
Adding annotator lemma


Name: org.apache.spark.sql.AnalysisException
Message: cannot resolve '`tfidVec`' given input columns: [title, terms, termFreqs, tfidfVec];;
'Project [title#38, 'tfidVec]
+- AnalysisBarrier
      +- Project [title#38, terms#39, termFreqs#59, UDF(termFreqs#59) AS tfidfVec#95]
         +- Project [title#38, terms#39, UDF(terms#39) AS termFreqs#59]
            +- Filter (size(terms#39) > 1)
               +- Project [_1#36 AS title#38, _2#37 AS terms#39]
                  +- SerializeFromObject [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, assertnotnull(assertnotnull(input[0, scala.Tuple2, true]))._1, true, false) AS _1#36, mapobjects(MapObjects_loopValue0, MapObjects_loopIsNull0, ObjectType(class java.lang.String), staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, lambdavariable(MapObjects_loopValue0, MapObjects_loopIsNull0, ObjectType(class java.lang.String), true), true, false), assertnotnull(assertnotnull(input[